In [ ]:
#Install the necessary packages

!pip install yfinance
!pip install matplotlib==3.5.3

In [ ]:
#Standard packages
import numpy as np
import pandas as pd

#Dates
from datetime import datetime, timedelta

#Finance packages
import yfinance as yf

#Optimization
import cvxpy as cp

#Plotting packages
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from matplotlib import rcParams

rcParams["font.size"] = 20
rcParams["axes.labelsize"] = 30

rcParams["xtick.labelsize"] = 16
rcParams["ytick.labelsize"] = 16

rcParams["figure.figsize"] = (8,6)

# Portfolio Optimization: Modern Portfolio Theory (Mean-Variance Analysis)

Modern Portfolio Theory (MPT) is an investment theory developed by Harry Markowitz in the 1950s. It is based on the idea that investors can maximize their expected return on investment by diversifying their portfolio across a range of assets. According to MPT, the expected return on a portfolio is determined by the combination of the individual returns of the assets within the portfolio and their corresponding risk levels. The risk of an asset is measured by its standard deviation, which is a measure of the amount of variation in the asset's return. MPT also introduces the concept of the efficient frontier, which is a curve that represents the portfolios that offer the highest expected return for a given level of risk. Portfolios that lie on the efficient frontier are considered to be optimal, as they offer the highest expected return for a given level of risk. MPT has been influential in the field of finance and has been widely adopted by investors and financial managers. However, it has also been criticized for its assumptions about investor behavior and its reliance on historical data.

**Assumptions**:

- Investors are rational and aim to maximize their expected return.
- Investors are risk averse and seek to minimize their risk.
- Asset returns are approximately normally distributed$^*$ 

$^*$This is indeed not true. The real assumption is that the *utility* of the investor can be well approximated by a quadratic form, which allows to be described by a function of only the first and second moments of the return distribution. But saying that returns are normally distributed is easier.

Under these assumptions, MPT introduces several key concepts and formulas to help investors create an optimal portfolio. We consider $N$ risky assets in which the investor might or not invest. The goal is then to obtain the proportion of the investor's money, or weight, $\omega_i$, that he/she should invest in each asset $i$. Thus, a portfolio is defined by a unique set of weights $\{\omega\}$, or in a more useful and precise way, by a vector $\vec{\omega}$ in which, as previously said, $\omega_i$ indicates the proportion of the investor's wealth allocated in each asset $i$. Being this definition clear, let $\vec{R}$ be the return matrix of the considered assets in which the element $r_{ij}$ is the return of asset $i$ at time $j$. Thus, the components of the mean return vector, $\vec{\mu}$, of the considered assets are defined as $\mu_i=\frac{1}{t}\sum_{j=1}^tr_{ij}$. With this information, we define the following statistical measures over a given portfolio:

<ins>**Expected Return**</ins>

The expected return of a portfolio is calculated as the sum of the expected returns of each asset multiplied by their respective weights in the portfolio:

$$ \mu_p=E[R_p] = \sum_{i=1}^N w_iE[R_i]=\vec{\omega}^T \vec{\mu} $$

where:

- $\mu_P$ is the expected return of the portfolio
- $w_i$ is the weight of asset $i$ in the portfolio
- $E[R_i]$ is the expected return of asset $i$
- $N$ is the number of assets in the portfolio

<ins>**Standard Deviation**</ins>

The standard deviation of a portfolio is calculated as the square root of the sum of the covariance between each pair of assets multiplied by their respective weights:

$$ \sigma_P=\sqrt{V[R_p]} = \sqrt{\sum_{i=1}^N\sum_{i=1, j=1}^N w_i\,\Sigma\,w_j}=\sqrt{\vec{\omega}^T\,\vec{\Sigma}\,\vec{\omega}}$$

where:

- $\sigma_P$ is the standard deviation of the portfolio
- $w_i$ is the weight of asset $i$ in the portfolio
- $\Sigma=Cov(R_i,R_j)$ is the covariance between assets $i$ and $j$
- $N$ is the number of assets in the portfolio

<ins>**Standard Deviation**</ins>

The standard deviation of a portfolio is the square root of the variance:

$$ \sigma_p = \sqrt{\sigma_P^2} $$

where:

- $\sigma_p$ is the standard deviation of the portfolio
- $\sigma_P^2$ is the variance of the portfolio

<ins>**Sharpe Ratio**</ins>

The Sharpe ratio is a measure of the risk-adjusted return of a portfolio. It is calculated as the excess return (the difference between the expected return and the risk-free rate) divided by the standard deviation:

$$ SR = \frac{\mu_P - R_f}{\sigma_p} $$

where:

- $SR$ is the Sharpe ratio
- $\mu_P$ is the expected return of the portfolio
- $R_f$ is the risk-free rate
- $\sigma_p$ is the standard deviation of the portfolio

To obtain the optimal portfolio we have to **maximize** the following objective function

$$f(\lambda, \vec{\omega})=(1-\lambda)\mu_P - \lambda \sigma_P^2 \quad \textrm{with} \quad \lambda\in(0,1)$$

or, equivalently, **minimize** the following one

$$f(\lambda, \vec{\omega})=\lambda\sigma^2_P - (1-\lambda)\mu_P \quad \textrm{with} \quad \lambda\in(0,1)$$

subject to some constraints:

- $\sum_i\omega_i=1$, this is, all money considered must be allocated.
- $w_i>0$ if we don't want to allow for short selling.

and some other restrictions could be added.

The $\lambda$ parameter is called the investor's risk aversion. Note that if $\lambda=0$ (the investor is not risk-averse), the optimization problem reduces to maximizing the expected return regardless of the risk involved. On the other hand, for $\lambda=1$ (most risk-averse investor, does not want "any" risk), the problem reduces to minimze the variance regardless of the expected return.

Solving the optimization problem for several values of $\lambda\in(0,1)$ we can compute the **Efficient Frontier**, which basically depicts the portfolios maximizing the returns for every predefined value of risk.

# The cvxpy library for solving *Convex Optimization Problems* relying on Disciplined Convex Programming (DCP)

CVXPY is an open source Python-embedded modeling language for convex optimization problems. It lets you express your problem in a natural way that follows the math, rather than in the restrictive standard form required by solvers.

Disciplined convex programming (DCP) is a system for constructing mathematical expressions with known curvature from a given library of base functions. CVXPY uses DCP to ensure that the specified optimization problems are convex.

Check it out at: https://www.cvxpy.org/

# Exercise 1. Classical (Markowitz) portfolio optimization

Implement the Mean-Variance model without allowing for short selling. This reduces to the following optimization problem:

\begin{equation}
\begin{array}{ll} \mbox{maximize} & (1-\lambda)\mu^T w - \lambda w^T\Sigma w\\
\mbox{subject to} & \sum_i w_i = 1, \quad w_i > 0 \ \forall i,
\end{array}
\end{equation}

**1. Download the data from several stocks and compute the log returns**

In [ ]:
start = datetime(2010, 1, 1)
end = datetime(2018, 1, 1)

#Apple, Microsoft, Amazon, Google, Nvidia, Telefonica, Santander, BBVA, Endesa
assets = ["AAPL", "MSFT", "AMZN", "GOOGL", "NVDA", "TEF.MC", "SAN.MC", "BBVA.MC", "ELE.MC"] 

df = yf.download(assets, start=start, end=end, interval="1d", progress=False, ignore_tz=True)["Adj Close"]

df_returns = np.log(1+df.pct_change()).dropna()

**2. Set the problem to be solved by DCP**

I provide the code for you, but look it carefully and try to understand it. Later you will have to implement it with some slight modifications.

In [ ]:
#Define and set the number of assets under consideration
N = len(df.columns)

#Define and set the mean return of the assets
mu = df_returns.mean().values

#Define and set the covariance matrix of the returns
Sigma = df_returns.cov().values

#Initialize the weight vector as cp.Variable(N) of the cvxpy framework
w = cp.Variable(N)

#Investor risk aversion parameter
risk_aversion = cp.Parameter(nonneg=True)

#Define how the mean return of the portfolio is computed: ret = w^T * mu
mean_P = mu.T @ w #(@ performs matrix multiplication in Python)

#Define how the variance of the portfolio is computed,  w^T*mu*w, which is a quadratic form
var_P = cp.quad_form(w, Sigma)

#Define expression to be solved
expression = cp.Maximize((1-risk_aversion)*mean_P - risk_aversion*var_P)

##Define the constraints
constraints = [cp.sum(w)==1, w>=0]

#Define the problem
prob = cp.Problem(expression, constraints)

**3. Solving the problem for a given risk aversion value**

Here I show you how to solve the defined optimization problem and obtain the optimal values. Check it out and play a little bit with the risk aversion parameter.

In [ ]:
#Set the value for the parameter
risk_aversion.value = 0.1

#Solve the problem
prob.solve()

#Obtain the optimal values
opt_ret = mean_P.value
opt_var = var_P.value
opt_w = w.value

print("Optimal annualized returns:", opt_ret*252)
print("Optimal annualized volatility:", np.sqrt(opt_var*252))
print("Sharpe Ratio:", opt_ret*252/np.sqrt(opt_var*252))
print("Optimal weights:\n", opt_w.reshape(-1,1))

**4. Compute the efficient frontier**

- Plot the efficient frontier as a scatter plot coloured according to the Sharpe Ratio. Plot together the individual stocks.

- Plot the mean value of the weight for each company. What do you observe?

In [ ]:
#CODE

**5. Save the weights (in a variable) of the optimal portfolio (the one yielding highest Sharpe Ratio)**

In [ ]:
#CODE

#Exercise 2. Portfolio optimization allowing for short-selling

Now we relax our constraint $w_i>0 \ \forall i$. Thus, negative weights are understood as fractions of the money devoted to short selling a particular stock.

**1. Define the problem**

In [ ]:
#CODE

**2. Compute and plot the efficient frontier**

In [ ]:
#CODE

**3. Save the weights (in a variable) of the optimal portfolio (the one yielding highest Sharpe Ratio)**

In [ ]:
#CODE

# Exercise 3. Future performance of optimal portfolios

1. Implement the optimal portfolios from the previous models (the ones yielding highest Sharpe Ratio) and check their performance in the period 2018-2023.

2. Implement the Equally Weighted (EW) portfolio and compare it to the other two.

  In the EW portfolio the weights are simply given by $w_i=1/N$.

3. Make a detailed analysis of the performance of the three portfolios. Compute:

  * Annualized return
  * Annualized volatility
  * Sharpe Ratio
  * Maximum Drawdown (MDD)

    A maximum drawdown (MDD) is the maximum observed loss from a peak to a trough of a portfolio, before a new peak is attained

    $$MDD(T)=max_{\tau\in(0,T)}D(\tau)=max_{\tau\in(0,T)}\left[max_{t\in(0,\tau)}X(t)-X(\tau)\right]$$

    Put it simple: the maximum loss from a peak to a subsequent trough in a given time period. It is usually computed as a percentage

    $$MDD=\frac{Peak-Trough}{Peak}\cdot 100$$

  * Return over Maximum Drawdown (RoMaD)

    The RoMaD is computed as the average return divided by the MDD in a given period

    $$RoMaD(T)=\frac{\left<R\right>_T}{MDD(T)}$$

**1. Download the data**

In [ ]:
start = datetime(2018, 1, 1)
end = datetime(2023, 1, 1)

#Apple, Microsoft, Amazon, Google, Nvidia, Telefonica, Santander, BBVA, Endesa
assets = ["AAPL", "MSFT", "AMZN", "GOOGL", "NVDA", "TEF.MC", "SAN.MC", "BBVA.MC", "ELE.MC"] 

df = yf.download(assets, start=start, end=end, interval="1d", progress=False, ignore_tz=True)["Adj Close"].dropna()

df_returns = np.log(1+df.pct_change()).dropna()

**2. Plot the evolution of the price of each portfolio**

In [ ]:
#CODE

**3 Detailed analysis**

In [ ]:
#CODE